In [4]:
# ===================================================================
# 【最終版 v4】只保存數據的精準抽樣腳本
#
# 更新日誌：
# 1. 根據用戶要求，移除保存標籤檔案的步驟，最終只輸出一個
#    inference_data.pt 檔案。
# ===================================================================

import numpy as np
import pandas as pd
import torch
import os
import gc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from tqdm import tqdm

# --- 1. 請設定您的數據集路徑 ---
DATASET_PATH = '/kaggle/input/cicids2017'

# --- 2. 請設定抽樣數量 ---
NUM_BENIGN_SAMPLES = 120
NUM_SAMPLES_PER_ATTACK_CLASS = 10
# 稀有類別的判斷閾值 (與原始訓練碼保持一致)
RARE_LABEL_THRESHOLD = 1000


def load_real_data(path):
    """從指定路徑加載所有 Parquet 檔案"""
    print(f"--- 正在從路徑讀取真實數據: {path} ---")
    if not os.path.exists(path) or not os.path.isdir(path):
        raise FileNotFoundError(f"錯誤：找不到數據路徑 '{path}'。")
    parquet_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.parquet')]
    if not parquet_files:
        raise FileNotFoundError(f"錯誤：在 '{path}' 中找不到 .parquet 檔案。")
    df_list = [pd.read_parquet(file) for file in tqdm(parquet_files, desc="正在載入 Parquet 檔案")]
    df = pd.concat(df_list, ignore_index=True)
    df.rename(columns={col: col.strip() for col in df.columns}, inplace=True)
    print(f"✓ 真實數據載入完成，原始形狀: {df.shape}")
    return df

def preprocess_and_group_labels(df):
    """
    對 DataFrame 進行清洗，並執行與原始訓練碼完全一致的標籤處理邏輯。
    """
    print("--- 正在進行數據預處理與標籤合併...")
    label_column = 'Label'
    if label_column not in df.columns:
        raise ValueError("在數據中找不到 'Label' 欄位。")
    
    # 統一格式，強制轉為大寫，並清理特殊字符
    df[label_column] = df[label_column].str.replace(r'[^a-zA-Z0-9\s-]', '', regex=True).str.strip().str.upper()
    # 處理 WEB ATTACK 中間多個空格的問題
    df[label_column] = df[label_column].str.replace(r'WEB ATTACK\s+', 'WEB ATTACK - ', regex=True)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    initial_rows = df.shape[0]
    df.drop_duplicates(inplace=True)
    print(f"移除了 {initial_rows - df.shape[0]} 筆重複記錄。")
    df.reset_index(drop=True, inplace=True)

    # 【關鍵】恢復原始訓練腳本的「稀有類別合併」邏輯
    y = df[label_column]
    label_counts = y.value_counts()
    rare_labels = label_counts[label_counts < RARE_LABEL_THRESHOLD].index
    
    rare_labels = [label for label in rare_labels if label != 'BENIGN']

    if rare_labels:
        print(f"檢測到 {len(rare_labels)} 個稀有類別 (樣本數 < {RARE_LABEL_THRESHOLD})，將合併為 'RARE_ATTACK'。")
        print(f"稀有類別列表: {rare_labels}")
        df[label_column].replace(rare_labels, 'RARE_ATTACK', inplace=True)

    print("✓ 數據預處理與標籤合併完成。")
    return df, label_column

def main():
    try:
        # 1. 載入並執行包含「稀有類別合併」的預處理
        full_df, label_column = preprocess_and_group_labels(load_real_data(DATASET_PATH))

        # 2. 獲取最終的類別列表以供抽樣
        final_classes = sorted(full_df[label_column].unique())
        print("\n========================= 最終類別資訊 =========================")
        print("經過稀有類別合併後，最終用於抽樣的類別有:")
        for label in final_classes:
            print(f"- {label}")
        print("=================================================================")

        # 3. 設定數據處理規則 (使用【全部】數據來擬合)
        print("\n--- 正在使用【全部數據】來設定數據處理規則...")
        X_full = full_df.drop(columns=[label_column])
        y_full = full_df[label_column]
        label_encoder = LabelEncoder().fit(y_full)
        var_selector = VarianceThreshold(threshold=0.001).fit(X_full)
        X_full_filtered = var_selector.transform(X_full)
        K_FEATURES = min(64, X_full_filtered.shape[1])
        selector = SelectKBest(f_classif, k=K_FEATURES).fit(X_full_filtered, label_encoder.transform(y_full))
        X_full_selected = selector.transform(X_full_filtered)
        scaler = StandardScaler().fit(X_full_selected)
        print("✓ 所有數據處理規則設定完成。")
        
        # 4. 按最終類別列表進行全自動精準抽樣
        print("\n--- 正在按最終類別列表和數量進行精準抽樣...")
        collected_samples = []
        
        for class_name in final_classes:
            num_to_sample = NUM_BENIGN_SAMPLES if class_name == 'BENIGN' else NUM_SAMPLES_PER_ATTACK_CLASS
            class_df = full_df[full_df[label_column] == class_name]
            
            if len(class_df) < num_to_sample:
                print(f"警告：'{class_name}' 類別數據不足 {num_to_sample} 筆 (只有 {len(class_df)} 筆)，將使用所有可用數據。")
                sampled_df = class_df
            else:
                sampled_df = class_df.sample(n=num_to_sample, random_state=42)
            
            collected_samples.append(sampled_df)
            print(f"  ✓ 已抽取 '{class_name}' 類別數據 {len(sampled_df)} 筆。")

        # 5. 組合、處理並存檔
        df_for_inference = pd.concat(collected_samples, ignore_index=True)
        df_for_inference = df_for_inference.sample(frac=1, random_state=42).reset_index(drop=True)
        
        print(f"\n--- 正在處理抽出的 {len(df_for_inference)} 筆樣本...")
        X_inference = df_for_inference.drop(columns=[label_column])
        
        # 雖然不保存標籤，但在處理過程中仍然需要它們
        # y_inference = df_for_inference[label_column]

        # y_inference_encoded = label_encoder.transform(y_inference) # 不再需要
        X_inference_filtered = var_selector.transform(X_inference)
        X_inference_selected = selector.transform(X_inference_filtered)
        X_inference_scaled = scaler.transform(X_inference_selected)
        
        X_inference_final = np.nan_to_num(X_inference_scaled, nan=0.0, posinf=1.0, neginf=-1.0)
        inference_tensor = torch.from_numpy(X_inference_final).float()
        print(f"✓ 樣本處理完成！最終數據形狀: {inference_tensor.shape}")

        # 6. 【修改處】只保存特徵數據，不保存標籤
        output_data_path = 'inference_data.pt'
        torch.save(inference_tensor, output_data_path)
        
        print(f"\n🎉 成功！已將 {len(df_for_inference)} 筆精準抽樣的數據保存至：")
        print(f"   - 特徵數據: {os.path.abspath(output_data_path)}")

    except (FileNotFoundError, ValueError) as e:
        print(f"\n❌ 操作失敗：{e}")
    except Exception as e:
        print(f"\n❌ 發生未預期的錯誤: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

--- 正在從路徑讀取真實數據: /kaggle/input/cicids2017 ---


正在載入 Parquet 檔案: 100%|██████████| 8/8 [00:01<00:00,  5.84it/s]


✓ 真實數據載入完成，原始形狀: (2313810, 78)
--- 正在進行數據預處理與標籤合併...
移除了 82004 筆重複記錄。
檢測到 4 個稀有類別 (樣本數 < 1000)，將合併為 'RARE_ATTACK'。
稀有類別列表: ['WEB ATTACK - XSS', 'INFILTRATION', 'WEB ATTACK - SQL INJECTION', 'HEARTBLEED']
✓ 數據預處理與標籤合併完成。

========================= 最終類別資訊 =========================
經過稀有類別合併後，最終用於抽樣的類別有:
- BENIGN
- BOT
- DDOS
- DOS GOLDENEYE
- DOS HULK
- DOS SLOWHTTPTEST
- DOS SLOWLORIS
- FTP-PATATOR
- PORTSCAN
- RARE_ATTACK
- SSH-PATATOR
- WEB ATTACK - BRUTE FORCE

--- 正在使用【全部數據】來設定數據處理規則...
✓ 所有數據處理規則設定完成。

--- 正在按最終類別列表和數量進行精準抽樣...
  ✓ 已抽取 'BENIGN' 類別數據 120 筆。
  ✓ 已抽取 'BOT' 類別數據 10 筆。
  ✓ 已抽取 'DDOS' 類別數據 10 筆。
  ✓ 已抽取 'DOS GOLDENEYE' 類別數據 10 筆。
  ✓ 已抽取 'DOS HULK' 類別數據 10 筆。
  ✓ 已抽取 'DOS SLOWHTTPTEST' 類別數據 10 筆。
  ✓ 已抽取 'DOS SLOWLORIS' 類別數據 10 筆。
  ✓ 已抽取 'FTP-PATATOR' 類別數據 10 筆。
  ✓ 已抽取 'PORTSCAN' 類別數據 10 筆。
  ✓ 已抽取 'RARE_ATTACK' 類別數據 10 筆。
  ✓ 已抽取 'SSH-PATATOR' 類別數據 10 筆。
  ✓ 已抽取 'WEB ATTACK - BRUTE FORCE' 類別數據 10 筆。

--- 正在處理抽出的 230 筆樣本...
✓ 樣本處理完成！最終數據形狀: torch.Size([230, 64])

🎉 成功！已